# Armoria API

In [1]:
import requests
import json
from PIL import Image
from io import BytesIO

class ArmoriaAPIWrapper:
    def __init__(self, size, format, coa):
        self.format = format
        payload = {"size": size, 
                   "format": format,
                   "coa": json.dumps(coa)
                  }
        self.r = requests.get('https://armoria.herokuapp.com/', params=payload)
#         print(self.r.url)
    
    def get_image_bytes(self):
        i = Image.open(BytesIO(self.r.content))
        return i
    
    def show_image(self):
        i = self.get_image_bytes()
        i.show()
    
    def save_image(self, full_path):
        i = self.get_image_bytes()    
        i.save(full_path, 'png')



In [2]:
coa_lion = {"t1":"or",
       "shield":"heater",
       "charges":[{"charge":"lionRampant",
                   "t":"gules",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_lion)
api.show_image()

coa_cross = {"t1":"argent",
       "shield":"heater",
       "charges":[{"charge":"crossMoline",
                   "t":"azure",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_cross)
api.show_image()


display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/432.


# Map our dataset to the API keywords

In [3]:
LION_MODIFIERS_MAP = {
    'lion rampant': 'lionRampant',
    'lion passt': 'lionPassant',
    'lion passt guard': 'lionPassantGuardant',
    "lion's head": 'lionHeadCaboshed'
}

CROSS_MODIFIERS_MAP = {
    'cross': 'crossHummetty' ,
    'cross moline': 'crossMoline',
    'cross patonce': 'crossPatonce',
}

    
EAGLE_MODIFIERS_MAP = {
    'eagle': 'eagle' ,
    'eagle doubleheaded': 'eagleTwoHeards',
}


MODIFIERS_MAP = {**LION_MODIFIERS_MAP, **CROSS_MODIFIERS_MAP, **EAGLE_MODIFIERS_MAP}

COLORS_MAP = { 'A': 'argent', # silver
              'B': 'azure', # blue
              'O': 'or', # gold 
              'S': 'sable', # black
              'G': 'gules', # red
              'V': 'vert' # green
             }
#               'E':'',
#               'X': '', 
#               'Z': ''}


# Armoria-API = possible values for single position of a charge in an image
p = ['a','b','c','d','e','f','g','h','i','y','z']

# Armoria-API = possible values for a charge scale 
# note that for large charges in a side positions means that a part of the charge is out of the shield
scale = ['0.5','1','1.5']

# Generate Permutations

In [4]:
def get_armoria_api_payload(label):
    shield_color = label[0][0]
    charge_color = label[0][1]
    charge = label[0][2]
    position = label[1]
    scale = label[2]
    
    api_shield_color = COLORS_MAP[shield_color]
    api_charge_color = COLORS_MAP[charge_color]
    api_charge = MODIFIERS_MAP[charge]

    coa = {"t1": api_shield_color, 
       "shield":"heater",
       "charges":[{"charge":api_charge,
                   "t": api_charge_color,
                   "p": position, 
                   "size":scale}] 
      }

    return coa


def save_label_txt(file_name, label):
    f = open(file_name ,"w+")
    f.write(' '.join(label))
    f.close()


def generate_dataset(folder_name, permutations):
    counter = 1

    # create a file to store all generated images files names and labels
    f = open(folder_name + '/' + 'captions.txt' ,"w+")
    f.write('image,caption')
    f.write('\n')

    for label in permutations:

        payload = get_armoria_api_payload(label)

        api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)

        sample_name = 'image_' + str(counter)

        image_full_path = folder_name + '/images/' + sample_name + '.png'
        text_label = ' '.join(label[0])
        print(label)

        api.save_image(image_full_path)

    #     caption_full_path = FOLDER_NAME + '/images/' + sample_name + ".txt"
    #     save_label_txt(caption_full_path, label)

        print('Image "{}" for label "{}" has been generated succfully' .format(image_full_path, text_label) )

        counter = counter + 1

        ## store image labels in on txt file (needed for data loader) 
        f.write(sample_name + '.png,' + text_label)
        f.write('\n')

    f.close()


display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/432.


In [5]:
import itertools
import src.alphabet as alphabet

all_possible_permutations = []

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( LION_MODIFIERS_MAP.keys()))))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()))))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( EAGLE_MODIFIERS_MAP.keys()))))

possible_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(possible_permutations))
# possible_permutations

Total number of permutations: 324


In [6]:
all_possible_permutations = list(dict.fromkeys(itertools.product(list(possible_permutations), p, scale)))
print('Total number of permutations with different position and scale:', len(all_possible_permutations))


Total number of permutations with different position and scale: 10692


In [7]:
all_possible_permutations[0]

(('A', 'A', 'lion rampant'), 'a', '0.5')

In [ ]:
FOLDER_NAME = '/home/space/datasets/COA/generated-data-api'
generate_dataset(FOLDER_NAME, all_possible_permutations)

(('A', 'A', 'lion rampant'), 'a', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_1.png" for label "A A lion rampant" has been generated succfully
(('A', 'A', 'lion rampant'), 'a', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_2.png" for label "A A lion rampant" has been generated succfully
(('A', 'A', 'lion rampant'), 'a', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_3.png" for label "A A lion rampant" has been generated succfully
(('A', 'A', 'lion rampant'), 'b', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_4.png" for label "A A lion rampant" has been generated succfully
(('A', 'A', 'lion rampant'), 'b', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_5.png" for label "A A lion rampant" has been generated succfully
(('A', 'A', 'lion rampant'), 'b', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_6.png" for label "A A lion rampant" has been gene

(('A', 'A', 'lion passt'), 'f', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_50.png" for label "A A lion passt" has been generated succfully
(('A', 'A', 'lion passt'), 'f', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_51.png" for label "A A lion passt" has been generated succfully
(('A', 'A', 'lion passt'), 'g', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_52.png" for label "A A lion passt" has been generated succfully
(('A', 'A', 'lion passt'), 'g', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_53.png" for label "A A lion passt" has been generated succfully
(('A', 'A', 'lion passt'), 'g', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_54.png" for label "A A lion passt" has been generated succfully
(('A', 'A', 'lion passt'), 'h', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_55.png" for label "A A lion passt" has been generated succfully
((

(('A', 'A', 'lion passt guard'), 'z', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_98.png" for label "A A lion passt guard" has been generated succfully
(('A', 'A', 'lion passt guard'), 'z', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_99.png" for label "A A lion passt guard" has been generated succfully
(('A', 'A', "lion's head"), 'a', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_100.png" for label "A A lion's head" has been generated succfully
(('A', 'A', "lion's head"), 'a', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_101.png" for label "A A lion's head" has been generated succfully
(('A', 'A', "lion's head"), 'a', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_102.png" for label "A A lion's head" has been generated succfully
(('A', 'A', "lion's head"), 'b', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_103.png" for label "A A lion's h

(('A', 'B', 'lion rampant'), 'e', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_147.png" for label "A B lion rampant" has been generated succfully
(('A', 'B', 'lion rampant'), 'f', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_148.png" for label "A B lion rampant" has been generated succfully
(('A', 'B', 'lion rampant'), 'f', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_149.png" for label "A B lion rampant" has been generated succfully
(('A', 'B', 'lion rampant'), 'f', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_150.png" for label "A B lion rampant" has been generated succfully
(('A', 'B', 'lion rampant'), 'g', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_151.png" for label "A B lion rampant" has been generated succfully
(('A', 'B', 'lion rampant'), 'g', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_152.png" for label "A B lion rampant" h

(('A', 'B', 'lion passt'), 'z', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_196.png" for label "A B lion passt" has been generated succfully
(('A', 'B', 'lion passt'), 'z', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_197.png" for label "A B lion passt" has been generated succfully
(('A', 'B', 'lion passt'), 'z', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_198.png" for label "A B lion passt" has been generated succfully
(('A', 'B', 'lion passt guard'), 'a', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_199.png" for label "A B lion passt guard" has been generated succfully
(('A', 'B', 'lion passt guard'), 'a', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_200.png" for label "A B lion passt guard" has been generated succfully
(('A', 'B', 'lion passt guard'), 'a', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_201.png" for label "A B lion pa

...

(('S', 'S', 'eagle doubleheaded'), 'z', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9767.png" for label "S S eagle doubleheaded" has been generated succfully
(('S', 'S', 'eagle doubleheaded'), 'z', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9768.png" for label "S S eagle doubleheaded" has been generated succfully
(('S', 'G', 'eagle'), 'a', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9769.png" for label "S G eagle" has been generated succfully
(('S', 'G', 'eagle'), 'a', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9770.png" for label "S G eagle" has been generated succfully
(('S', 'G', 'eagle'), 'a', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9771.png" for label "S G eagle" has been generated succfully
(('S', 'G', 'eagle'), 'b', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9772.png" for label "S G eagle" has been generated succfull

(('S', 'G', 'eagle doubleheaded'), 'f', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9817.png" for label "S G eagle doubleheaded" has been generated succfully
(('S', 'G', 'eagle doubleheaded'), 'f', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9818.png" for label "S G eagle doubleheaded" has been generated succfully
(('S', 'G', 'eagle doubleheaded'), 'f', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9819.png" for label "S G eagle doubleheaded" has been generated succfully
(('S', 'G', 'eagle doubleheaded'), 'g', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9820.png" for label "S G eagle doubleheaded" has been generated succfully
(('S', 'G', 'eagle doubleheaded'), 'g', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9821.png" for label "S G eagle doubleheaded" has been generated succfully
(('S', 'G', 'eagle doubleheaded'), 'g', '1.5')
Image "/home/space/datasets/COA/

(('S', 'V', 'eagle'), 'z', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9866.png" for label "S V eagle" has been generated succfully
(('S', 'V', 'eagle'), 'z', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9867.png" for label "S V eagle" has been generated succfully
(('S', 'V', 'eagle doubleheaded'), 'a', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9868.png" for label "S V eagle doubleheaded" has been generated succfully
(('S', 'V', 'eagle doubleheaded'), 'a', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9869.png" for label "S V eagle doubleheaded" has been generated succfully
(('S', 'V', 'eagle doubleheaded'), 'a', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9870.png" for label "S V eagle doubleheaded" has been generated succfully
(('S', 'V', 'eagle doubleheaded'), 'b', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9871.png" for label 

(('G', 'A', 'eagle'), 'e', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9913.png" for label "G A eagle" has been generated succfully
(('G', 'A', 'eagle'), 'e', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9914.png" for label "G A eagle" has been generated succfully
(('G', 'A', 'eagle'), 'e', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9915.png" for label "G A eagle" has been generated succfully
(('G', 'A', 'eagle'), 'f', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9916.png" for label "G A eagle" has been generated succfully
(('G', 'A', 'eagle'), 'f', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9917.png" for label "G A eagle" has been generated succfully
(('G', 'A', 'eagle'), 'f', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9918.png" for label "G A eagle" has been generated succfully
(('G', 'A', 'eagle'), 'g', '0.5')
Image "/home/spa

(('G', 'A', 'eagle doubleheaded'), 'y', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9961.png" for label "G A eagle doubleheaded" has been generated succfully
(('G', 'A', 'eagle doubleheaded'), 'y', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9962.png" for label "G A eagle doubleheaded" has been generated succfully
(('G', 'A', 'eagle doubleheaded'), 'y', '1.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9963.png" for label "G A eagle doubleheaded" has been generated succfully
(('G', 'A', 'eagle doubleheaded'), 'z', '0.5')
Image "/home/space/datasets/COA/generated-data-api/images/image_9964.png" for label "G A eagle doubleheaded" has been generated succfully
(('G', 'A', 'eagle doubleheaded'), 'z', '1')
Image "/home/space/datasets/COA/generated-data-api/images/image_9965.png" for label "G A eagle doubleheaded" has been generated succfully
(('G', 'A', 'eagle doubleheaded'), 'z', '1.5')
Image "/home/space/datasets/COA/

In [13]:
# Generate smaller dataset for local testing

import itertools
import src.alphabet as alphabet

all_possible_permutations = []

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(['A'],list(COLORS_MAP.keys()), list( LION_MODIFIERS_MAP.keys()))))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(['A'], list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()))))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(['A'], list(COLORS_MAP.keys()), list( EAGLE_MODIFIERS_MAP.keys()))))

all_possible_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(all_possible_permutations))
all_possible_permutations

Total number of permutations: 54


[('A', 'A', 'lion rampant'),
 ('A', 'A', 'lion passt'),
 ('A', 'A', 'lion passt guard'),
 ('A', 'A', "lion's head"),
 ('A', 'B', 'lion rampant'),
 ('A', 'B', 'lion passt'),
 ('A', 'B', 'lion passt guard'),
 ('A', 'B', "lion's head"),
 ('A', 'O', 'lion rampant'),
 ('A', 'O', 'lion passt'),
 ('A', 'O', 'lion passt guard'),
 ('A', 'O', "lion's head"),
 ('A', 'S', 'lion rampant'),
 ('A', 'S', 'lion passt'),
 ('A', 'S', 'lion passt guard'),
 ('A', 'S', "lion's head"),
 ('A', 'G', 'lion rampant'),
 ('A', 'G', 'lion passt'),
 ('A', 'G', 'lion passt guard'),
 ('A', 'G', "lion's head"),
 ('A', 'V', 'lion rampant'),
 ('A', 'V', 'lion passt'),
 ('A', 'V', 'lion passt guard'),
 ('A', 'V', "lion's head"),
 ('A', 'A', 'cross'),
 ('A', 'A', 'cross moline'),
 ('A', 'A', 'cross patonce'),
 ('A', 'B', 'cross'),
 ('A', 'B', 'cross moline'),
 ('A', 'B', 'cross patonce'),
 ('A', 'O', 'cross'),
 ('A', 'O', 'cross moline'),
 ('A', 'O', 'cross patonce'),
 ('A', 'S', 'cross'),
 ('A', 'S', 'cross moline'),
 ('A

In [15]:
FOLDER_NAME = 'generated-data-api-small'
generate_dataset(FOLDER_NAME, all_possible_permutations)

Image "generated-data-api-small/images/image_1.png" for label "A A lion rampant" has been generated succfully
Image "generated-data-api-small/images/image_2.png" for label "A A lion passt" has been generated succfully
Image "generated-data-api-small/images/image_3.png" for label "A A lion passt guard" has been generated succfully
Image "generated-data-api-small/images/image_4.png" for label "A A lion's head" has been generated succfully
Image "generated-data-api-small/images/image_5.png" for label "A B lion rampant" has been generated succfully
Image "generated-data-api-small/images/image_6.png" for label "A B lion passt" has been generated succfully
Image "generated-data-api-small/images/image_7.png" for label "A B lion passt guard" has been generated succfully
Image "generated-data-api-small/images/image_8.png" for label "A B lion's head" has been generated succfully
Image "generated-data-api-small/images/image_9.png" for label "A O lion rampant" has been generated succfully
Image "g